In [ ]:
!pip install langchain langchain-nvidia-ai-endpoints langchain-tavily langchain-community langgraph pydantic grandalf 

In [ ]:
from kaggle_secrets import UserSecretsClient
nvidia_api_key = UserSecretsClient().get_secret("NVIDIA_API_KEY")
nvidia_model_name = UserSecretsClient().get_secret("NVIDIA_MODEL_NAME")
tavily_api_key = UserSecretsClient().get_secret("TAVILY_API_KEY")

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.agents import tool, create_react_agent
import datetime
from langchain_tavily import TavilySearch
from langchain import hub

In [ ]:
llm = ChatNVIDIA(
    model = nvidia_model_name,
    api_key = nvidia_api_key,
)

In [ ]:
search_tool = TavilySearch(tavily_api_key = tavily_api_key, max_results = 5, topic = "general")

In [ ]:
from langchain_community.tools import TavilySearchResults

In [ ]:
search_tool_2 = TavilySearchResults(tavily_api_key = tavily_api_key)

In [ ]:
@tool
def get_system_time(format: str = "%Y-%m-%d %H-%M-%S"):
    """ Returns the current date and time in the specified format """

    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime(format)
    return formatted_time

In [ ]:
tools = [search_tool_2, get_system_time]

In [ ]:
react_prompt = hub.pull("hwchase17/react")

In [ ]:
print(react_prompt)

In [ ]:
react_agent_runnable = create_react_agent(
    tools = tools,
    llm = llm,
    prompt = react_prompt,
) 

## State Building

In [ ]:
from typing import Annotated, TypedDict, Union
import operator

from langchain_core.agents import AgentAction, AgentFinish

In [ ]:
class AgentState(TypedDict):
    input: str
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
    

## Nodes

In [ ]:
!pip install langgraph

In [ ]:
from langgraph.prebuilt import ToolNode

In [ ]:
tool_executor = ToolNode(tools)

In [ ]:
def reason_node(state: AgentState):
    agent_outcome = react_agent_runnable.invoke(state)
    return {"agent_outcome": agent_outcome}

In [ ]:
def act_node(state: AgentState):
    agent_action = state["agent_outcome"]

    # tool_name = agent_action.tool
    # tool_input = agent_action.tool_input

    # tool_function = None
    # for tool in tools:
    #     if tool.name == tool_name:
    #         tool_function = tool
    #         break

    # if tool_function:
    #     if isinstance(tool_input, dict):
    #         output = tool_function.invoke(**tool_input)
    #     else:
    #         output = tool_function.invoke(tool_input)
    
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}

## Building Graph

In [ ]:
from langchain_core.agents import AgentFinish, AgentAction
from langgraph.graph import END, StateGraph

In [ ]:
REASON_NODE = "reason_node"
ACT_NODE = "act_node"

In [ ]:
def should_continue(state: AgentState) -> str:
    if isinstance(state["agent_outcome"], AgentFinish):
        return END
    else:
        return ACT_NODE

In [ ]:
flow = StateGraph(AgentState)

In [ ]:
flow.add_node(REASON_NODE, reason_node)

In [ ]:
flow.set_entry_point(REASON_NODE)

In [ ]:
flow.add_node(ACT_NODE, act_node)

In [ ]:
flow.add_conditional_edges(
    REASON_NODE,
    should_continue,
)

In [ ]:
flow.add_edge(ACT_NODE, REASON_NODE)

In [ ]:
app = flow.compile()

In [ ]:
result = app.invoke({
    "input": "How many days ago was the latest SpaceX launch?",
    "agent_outcome": None,
    "intermediate_steps": []
})

In [ ]:
print(result)